In [1]:
!pip install tensorflow scikit-learn matplotlib seaborn


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os

# Proje ana dizinin (sen kendine göre değiştirebilirsin)
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/Sugarcane Disease"

In [7]:

# Ham veri klasörü Sugarcane Disease'in kendisi
RAW_DIR = BASE_DIR

# Ayırılmış dataset buraya oluşturulacak
SPLIT_DIR = os.path.join(BASE_DIR, "dataset_split")

print("BASE_DIR   :", BASE_DIR)
print("RAW_DIR    :", RAW_DIR)
print("SPLIT_DIR  :", SPLIT_DIR)

print("RAW klasörü var mı?", os.path.exists(RAW_DIR))


BASE_DIR   : /content/drive/MyDrive/Colab Notebooks/Sugarcane Disease
RAW_DIR    : /content/drive/MyDrive/Colab Notebooks/Sugarcane Disease
SPLIT_DIR  : /content/drive/MyDrive/Colab Notebooks/Sugarcane Disease/dataset_split
RAW klasörü var mı? True


In [8]:
classes = os.listdir(RAW_DIR)
print("Bulunan sınıf klasörleri:")
for c in classes:
    print("-", c)


Bulunan sınıf klasörleri:
- Mosaic
- RedRot
- Healthy
- Rust
- Yellow


In [9]:
import shutil
import random

# Oranlar
train_ratio = 0.7
val_ratio   = 0.1
test_ratio  = 0.2

assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Oranlar 1 etmiyor!"

# Tekrarlanabilirlik için
random.seed(42)

# dataset_split altındaki ana klasörler
for split in ["train", "val", "test"]:
    split_path = os.path.join(SPLIT_DIR, split)
    os.makedirs(split_path, exist_ok=True)

print("train/val/test klasörleri oluşturuldu/var.")


train/val/test klasörleri oluşturuldu/var.


In [10]:
# Yalnızca resim uzantılarını dikkate alalım
VALID_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp')

for cls in classes:
    class_path = os.path.join(RAW_DIR, cls)
    if not os.path.isdir(class_path):
        continue  # klasör değilse geç

    # Bu sınıfa ait tüm resimleri topla
    images = [
        f for f in os.listdir(class_path)
        if f.lower().endswith(VALID_EXTENSIONS)
    ]

    if len(images) == 0:
        print(f"UYARI: '{cls}' klasöründe resim bulunamadı!")
        continue

    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * train_ratio)
    n_val   = int(n_total * val_ratio)
    n_test  = n_total - n_train - n_val  # kalan test

    train_imgs = images[:n_train]
    val_imgs   = images[n_train:n_train + n_val]
    test_imgs  = images[n_train + n_val:]

    print(f"\nSınıf: {cls}")
    print(f"  Toplam: {n_total}")
    print(f"  Train:  {len(train_imgs)}")
    print(f"  Val:    {len(val_imgs)}")
    print(f"  Test:   {len(test_imgs)}")

    # Her split için hedef klasör ve kopyalama
    for split, split_imgs in zip(["train", "val", "test"],
                                 [train_imgs, val_imgs, test_imgs]):
        split_class_dir = os.path.join(SPLIT_DIR, split, cls)
        os.makedirs(split_class_dir, exist_ok=True)

        for img_name in split_imgs:
            src = os.path.join(class_path, img_name)
            dst = os.path.join(split_class_dir, img_name)
            shutil.copy2(src, dst)

print("\n✅ Veri başarıyla train/val/test olarak bölündü!")



Sınıf: Mosaic
  Toplam: 462
  Train:  323
  Val:    46
  Test:   93

Sınıf: RedRot
  Toplam: 518
  Train:  362
  Val:    51
  Test:   105

Sınıf: Healthy
  Toplam: 522
  Train:  365
  Val:    52
  Test:   105

Sınıf: Rust
  Toplam: 514
  Train:  359
  Val:    51
  Test:   104

Sınıf: Yellow
  Toplam: 505
  Train:  353
  Val:    50
  Test:   102

✅ Veri başarıyla train/val/test olarak bölündü!


In [11]:
def count_images(folder):
    total = 0
    for root, dirs, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(VALID_EXTENSIONS):
                total += 1
    return total

train_count = count_images(os.path.join(SPLIT_DIR, "train"))
val_count   = count_images(os.path.join(SPLIT_DIR, "val"))
test_count  = count_images(os.path.join(SPLIT_DIR, "test"))

print("Train toplam görüntü sayısı :", train_count)
print("Val   toplam görüntü sayısı :", val_count)
print("Test  toplam görüntü sayısı :", test_count)
print("Toplam (split) :", train_count + val_count + test_count)

raw_total = count_images(RAW_DIR)
print("Ham veri toplam görüntü sayısı:", raw_total)


Train toplam görüntü sayısı : 1762
Val   toplam görüntü sayısı : 250
Test  toplam görüntü sayısı : 509
Toplam (split) : 2521
Ham veri toplam görüntü sayısı: 5042
